In the previous tutorial we created a work queue. The assumption behind a work queue is that each task is delivered to exactly one worker. In this part we'll do something completely different -- we'll deliver a message to multiple consumers.  

To illustrate the pattern, we're going to build a simple logging system. It will consist of two programs -- the first will emit log messages and the second will receive and print them.

Essentially, published log messages are going to be broadcast to all the receivers.

### Exchanges
In previous parts of the tutorial we sent and received messages to and from a queue. Now it's time to introduce the full messaging model in Rabbit.

Let's quickly go over what we covered in the previous tutorials:

A producer is a user application that sends messages.
A queue is a buffer that stores messages.
A consumer is a user application that receives messages.
The core idea in the messaging model in RabbitMQ is that the producer never sends any messages directly to a queue. Actually, quite often the producer doesn't even know if a message will be delivered to any queue at all.

Instead, the producer can only send messages to an exchange. An exchange is a very simple thing. On one side it receives messages from producers and the other side it pushes them to queues. The exchange must know exactly what to do with a message it receives. Should it be appended to a particular queue? Should it be appended to many queues? Or should it get discarded. The rules for that are defined by the exchange type.

There are a few exchange types available: direct, topic, headers and fanout. We'll focus on the last one -- the fanout. Let's create an exchange of that type, and call it logs:

channel.exchange_declare(exchange='logs',exchange_type='fanout')

The fanout exchange is very simple. As you can probably guess from the name, it just broadcasts all the messages it receives to all the queues it knows. And that's exactly what we need for our logger.

### Listing exchanges
To list the exchanges on the server you can run the ever useful rabbitmqctl:

sudo rabbitmqctl list_exchanges

In this list there will be some amq.* exchanges and the default (unnamed) exchange. These are created by default, but it is unlikely you'll need to use them at the moment.

#### The default exchange
In previous parts of the tutorial we knew nothing about exchanges, but still were able to send messages to queues. That was possible because we were using a default exchange, which we identify by the empty string ("").

Recall how we published a message before:

channel.basic_publish(exchange='',
                      routing_key='hello',
                      body=message)
                      
The exchange parameter is the name of the exchange. The empty string denotes the default or nameless exchange: messages are routed to the queue with the name specified by routing_key, if it exists.

### Temporary queues

As you may remember previously we were using queues that had specific names (remember hello and task_queue?). Being able to name a queue was crucial for us -- we needed to point the workers to the same queue. Giving a queue a name is important when you want to share the queue between producers and consumers.

But that's not the case for our logger. We want to hear about all log messages, not just a subset of them. We're also interested only in currently flowing messages not in the old ones. To solve that we need two things.

Firstly, whenever we connect to Rabbit we need a fresh, empty queue. To do it we could create a queue with a random name, or, even better - let the server choose a random queue name for us. We can do this by not supplying the queue parameter to queue_declare:

result = channel.queue_declare()
At this point result.method.queue contains a random queue name. For example it may look like amq.gen-JzTY20BRgKO-HjmUJj0wLg.

Secondly, once the consumer connection is closed, the queue should be deleted. There's an exclusive flag for that:

result = channel.queue_declare(exclusive=True)

### Bindings

We've already created a fanout exchange and a queue. Now we need to tell the exchange to send messages to our queue. That relationship between exchange and a queue is called a binding.

channel.queue_bind(exchange='logs',
                   queue=result.method.queue)
From now on the logs exchange will append messages to our queue.

#### Listing bindings
You can list existing bindings using, you guessed it,

sudo rabbitmqctl list_bindings

In [10]:
import pika
import sys
pika.__version__

'1.0.0'

In [ ]:
#!sudo service rabbitmq-server restart
#!sudo rabbitmqctl status

In [11]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [12]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
    
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)

iter=0
exchangeName = 'logs'

In [20]:
iter=iter+1
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange=exchangeName, exchange_type='fanout')
#channel.queue_declare(queue='task_queue', durable=True)

#send message #for fanout type exchange, routing key is useless
properties = pika.spec.BasicProperties(content_type = "text/plain", delivery_mode = 1)
message = "MessageToEveryOne!"+str(iter)

channel.basic_publish(exchange = exchangeName, routing_key = "", body = message, properties = properties)

#disconnect
connection.close()

In [18]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	080bf8d3eacb4d418794f84218ab4c40	queue	080bf8d3eacb4d418794f84218ab4c40	[]
	exchange	32661066bdb14df482ef6bb2e13eb306	queue	32661066bdb14df482ef6bb2e13eb306	[]
	exchange	65e9bd76d110465a8cc8eb44f898bfc2	queue	65e9bd76d110465a8cc8eb44f898bfc2	[]
	exchange	amq.gen--6Yaw_vMGuJxLejx8SO6CA	queue	amq.gen--6Yaw_vMGuJxLejx8SO6CA	[]
	exchange	amq.gen-soDqMzeMYij2fMx5PHzquQ	queue	amq.gen-soDqMzeMYij2fMx5PHzquQ	[]
	exchange	b14ea0f1aad9462f85954bf0ff36753e	queue	b14ea0f1aad9462f85954bf0ff36753e	[]
	exchange	c9f98819d5974fac9b8d631f4622ab85	queue	c9f98819d5974fac9b8d631f4622ab85	[]
	exchange	celery	queue	celery	[]
	exchange	d5c923116f5644ddb5838ad52b128d5d	queue	d5c923116f5644ddb5838ad52b128d5d	[]
	exchange	da50811cd8664935b313824e18aed7f2	queue	da50811cd8664935b313824e18aed7f2	[]
	exchange	f9e709d96900462480e5b033c7e7cc1e	queue	f9e709d96900462480e5b033c7e7cc1e	[]
	exchange	task_queue	queue	task_queue	[]
celery	exchange	celery	queue	celery	[]
celeryresults	exchange	0

In [19]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
celery	direct
celeryev	topic
celeryresults	direct
logs	fanout
